In [5]:
import socket # 소켓 프로그래밍 용 라이브러리 import
import json # 정보 전달용 json import
import threading #프로세스를 담당하기 위한 treading import

HOST = '0.0.0.0' # host는 로컬
PORT = 12000 # 포트 번호는 12000
BUFFER_SIZE = 2048 # 버퍼 사이즈 2048byte로 사용

def process_message(username, message, option): #메세지 처리(선택 1,2,3,4)
    if option == 1: #1 이면 메세지 그대로 출력
        result = message
    elif option == 2: #2 이면 대문자로 바꿔서 출력
        result = message.upper()
    elif option == 3: #3 이면 소문자로 바꿔서 출력
        result = message.lower()
    elif option == 4: #4 이면 프로그램 종료
        return {'status': 200, 'message': f'[{username}]: goodbye'}, True #종료할 때 따로 print를 하는게 아니라 json에 종료 메세지를 넣어서 그대로 출력
    else:
        return {'status': 400 + option, 'error': 'Invalid echo option'}, False #이상한 값이 들어온다면 오류 메세지 출력
    return {'status': 200, 'message': f'[{username}]: {result}'}, False #반환

def start_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((HOST, PORT))
        server_socket.listen(5)
        print(f"[SERVER] Listening on port {PORT}...")

        while True:
            client_socket, addr = server_socket.accept()
            with client_socket:
                data = client_socket.recv(BUFFER_SIZE).decode()

                try:
                    request = json.loads(data)
                    username = request.get('user')
                    message = request.get('message')
                    option = int(request.get('echoOption'))

                    response, should_exit = process_message(username, message, option)
                    client_socket.sendall(json.dumps(response).encode())

                    print(f"[RECV] {request}")
                    print(f"[SEND] {response}")

                    if should_exit: 
                        break
                except Exception as e: #예외 처리
                    error_response = json.dumps({"status": 500, "error": "Invalid request format"})
                    client_socket.sendall(error_response.encode())
                    print(f"[ERROR] {e}")

# 스레드로 실행
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/gimhyeonjin/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure


    _threading_Thread_run(self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/xx/z516r8t944qclv1416b4lq3w0000gn/T/ipykernel_84288/868119538.py", line 24, in start_server
OSError: [Errno 48] Address already in use


In [6]:
import socket
import json

SERVER_HOST = '127.0.0.1'
SERVER_PORT = 12000
BUFFER_SIZE = 2048

username = input("사용자 이름: ")

while True:
    try:
        option = int(input("옵션 (1: 그대로, 2: 대문자, 3: 소문자, 4: 종료): "))
        if option < 1 or option > 4:
            print("[ERROR] 옵션은 1~4 사이의 정수여야 합니다.")
            continue
    except ValueError:
        print("[ERROR] 숫자를 입력해주세요.")
        continue

    if option == 4:
        print("[INFO] 종료 요청")
        break

    message = input("메시지 입력: ")

    packet = { #json 형태로 입력
        "user": username,
        "echoOption": option,
        "message": message
    }

    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
            client_socket.connect((SERVER_HOST, SERVER_PORT))
            client_socket.sendall(json.dumps(packet).encode())

            response = client_socket.recv(BUFFER_SIZE).decode()
            response_data = json.loads(response)

            if "message" in response_data:
                print("응답:", response_data["message"])
            elif "error" in response_data:
                print(f"오류 {response_data['status']}: {response_data['error']}")
    except Exception as e:
        print(f"[ERROR] 서버 연결 실패: {e}")

응답: 채주혁]: 바보
응답: 채주혁]: TEST
응답: 채주혁]: dfsfdsa
[INFO] 종료 요청
